In [2]:
import pandas as pd
import lucem_illud
import nltk
import matplotlib.pyplot as plt
import seaborn

In [8]:
def wordCounter(wordLst):
    wordCounts = {}
    for word in wordLst:
        #We usually need to normalize the case
        wLower = word.lower()
        if wLower in wordCounts:
            wordCounts[wLower] += 1
        else:
            wordCounts[wLower] = 1
    #convert to DataFrame
    countsForFrame = {'word' : [], 'count' : []}
    for w, c in wordCounts.items():
        countsForFrame['word'].append(w)
        countsForFrame['count'].append(c)
    return pd.DataFrame(countsForFrame)

In [16]:
bookdf = pd.read_csv("bookdf.csv", index_col=0)
bookdf['tokens'] = bookdf['text'].apply(lambda x: nltk.word_tokenize(x))
bookdf['nltktext'] = bookdf['tokens'].apply(lambda x: nltk.Text(x))

In [11]:
alltokens = bookdf['tokens'].sum()
allcountedWords = wordCounter(alltokens)
allcountedWords.sort_values('count', ascending=False, inplace=True)
stop_words_freq = []
countsDict = {}
for word in alltokens:
    if word in countsDict:
        countsDict[word] += 1
    else:
        countsDict[word] = 1
word_counts = sorted(countsDict.items(), key = lambda x : x[1], reverse = True)
word_counts[:100]
for word, count in word_counts:
    if word == 'little':
        break
    else:
        stop_words_freq.append(word)

In [15]:
bookdf['tokenized_sents'] = bookdf['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])
bookdf['normalized_sents'] = bookdf['tokenized_sents'].apply(lambda x: [lucem_illud.normalizeTokens(s, stopwordLst = stop_words_freq, stemmer = None) for s in x])
bookdf[:10]

,author,text,title,tokens,nltktext,tokenized_sents,normalized_sents
0,Jules Feiffer,"George’s mother said: ""Bark, George."" George w...","Bark, George","[George, ’, s, mother, said, :, ``, Bark, ,, G...","(George, ’, s, mother, said, :, ``, Bark, ,, G...","[[George, ’, s, mother, said, :, ``, Bark, ,, ...","[[george, mother, bark, george], [george, went..."
1,Robert McCloskey,"One day, Little Sal went with her mother to Bl...",Blueberries for Sal,"[One, day, ,, Little, Sal, went, with, her, mo...","(One, day, ,, Little, Sal, went, with, her, mo...","[[One, day, ,, Little, Sal, went, with, her, m...","[[one, day, little, sal, went, mother, blueber..."
2,Bill Martin Jr,"Brown Bear, Brown Bear, What do you see? I see...","Brown Bear, Brown Bear, What Do You See?","[Brown, Bear, ,, Brown, Bear, ,, What, do, you...","(Brown, Bear, ,, Brown, Bear, ,, What, do, you...","[[Brown, Bear, ,, Brown, Bear, ,, What, do, yo...","[[brown, bear, brown, bear, what, do, see], [i..."
3,Doreen Cronin,Farmer Brown has a problem. His cows like to t...,"Click, Clack, Moo Cows that Type","[Farmer, Brown, has, a, problem, ., His, cows,...","(Farmer, Brown, has, a, problem, ., His, cows,...","[[Farmer, Brown, has, a, problem, .], [His, co...","[[farmer, brown, has, problem], [cows, type], ..."
4,Judi Barrett,We were all sitting around the big kitchen tab...,Cloudy With a Chance of Meatballs,"[We, were, all, sitting, around, the, big, kit...","(We, were, all, sitting, around, the, big, kit...","[[We, were, all, sitting, around, the, big, ki...","[[we, were, sitting, around, big, kitchen, tab..."
5,Don Freeman,Corduroy is a bear who once lived in the toy d...,Corduroy,"[Corduroy, is, a, bear, who, once, lived, in, ...","(Corduroy, is, a, bear, who, once, lived, in, ...","[[Corduroy, is, a, bear, who, once, lived, in,...","[[corduroy, bear, who, once, lived, toy, depar..."
6,H.A. Rey,This is George. He lived in Africa. He was a g...,Curious George,"[This, is, George, ., He, lived, in, Africa, ....","(This, is, George, ., He, lived, in, Africa, ....","[[This, is, George, .], [He, lived, in, Africa...","[[this, george], [lived, africa], [good, littl..."
7,Sheila McGraw,A mother held her new baby and very slowly roc...,Love You Forever,"[A, mother, held, her, new, baby, and, very, s...","(A, mother, held, her, new, baby, and, very, s...","[[A, mother, held, her, new, baby, and, very, ...","[[mother, held, new, baby, very, slowly, rocke..."
8,Ruth Krauss,A little boy planted a carrot seed. His mother...,The Carrot Seed,"[A, little, boy, planted, a, carrot, seed, ., ...","(A, little, boy, planted, a, carrot, seed, ., ...","[[A, little, boy, planted, a, carrot, seed, .]...","[[little, boy, planted, carrot, seed], [mother..."
9,Ezra Jack Keats,One winter morning Peter woke up and looked ou...,The snowy Day,"[One, winter, morning, Peter, woke, up, and, l...","(One, winter, morning, Peter, woke, up, and, l...","[[One, winter, morning, Peter, woke, up, and, ...","[[one, winter, morning, peter, woke, looked, w..."
